# LLM to Motion

**Read the request documentation to tune your model for your application**

**Request Documentation**: https://platform.openai.com/docs/api-reference/completions/create

**OpenAI Documentation**: https://platform.openai.com/docs/quickstart?context=python

**Find your keys here**: https://platform.openai.com/api-keys

**Keep an eye on your credit usage**: https://platform.openai.com/usage

## OpenAI API Details and Hyper-Parameters

**Models**: The model you want to use (i.e. "gpt-4-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini" or "gpt-3.5-turbo")

**Message**: The message being sent (the prompt)

**Temperature**: Number between 0 and 1, can be adjusted when calling the OpenAI API to control the randomness of the output generated by the language model. It determines how "creative" or "conservative" the responses are. A temperature of 1.0 means the model generates text with standard randomness, while a lower temperature (closer to 0.0) will make it more deterministic. Suggestion: Set the temperature to a moderate value, around 0.2 to 0.5. This will give you some randomness, mimicking the variability in human input, but still maintain enough accuracy to ensure that the responses are meaningful. **In the case of our work, we held this constant at a value of 0.2 to get slight stochasticity.** 

**Frequency_penalty**: Number between -2 and 2, where higher numbers penalize new tokens based on their frequency to that point in the response. The higher the number, the lower the probability of repetition. This parameter penalizes repeated tokens in the output, making responses more diverse and creative. Suggestion: Use a low frequency penalty (e.g., 0.0 to 0.2) to reduce redundancy but still allow the model to use relevant tokens multiple times where necessary. **In the case of our work, we held this constant at a value of 0.0.**

## Set your GPT Key to your ENV

Uncomment this line to check your environment variables. That should be set up in .zshrc or .bashrc

In [12]:
# %env

In [13]:
%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_03/LLM_motion/scripts'

## Imports

In [14]:
import os
from openai import OpenAI
client = OpenAI()

import random
import time
random.seed(time.time())

from openpyxl import load_workbook

## Start Generating

### Build your prompt

In [15]:
gpt_assistant_prompt = "You are an expert roboticist and understand how to design communicative expressions for human-robot interaction." 

robot_morphology = "dog-shaped quadruped"
robot_state_name = "needs help"
robot_state_description = "the robot is experiencing an error and needs help from the user"

parameter_01 = "Parameter 01: [Body Tilt, Controls the left-right tilt angle of the robot's torso. If set to ‘left’ robot’s torso tilts to the left. If set to ‘neutral’ robot’s torso remains unchanged. If set to ‘right’ robot’s torso tilts to the right., (left, neutral, right)]"
parameter_02 = "Parameter 02: [Body Lean, Controls the forwards-backwards lean angle of the robot's torso. If set to ‘backwards’ robot’s torso leans backwards. If set to ‘neutral’ robot’s torso remains unchanged. If set to ‘forwards’ robot’s torso leans forwards.,(backwards, neutral, forwards)]"
parameter_03 = "Parameter 03: [Body Turn, Controls the left-right turn angle of the robot's torso. If set to ‘left’ robot’s torso turns to the left. If set to ‘neutral’ robot’s torso remains unchanged. If set to ‘right’ robot’s torso turns to the right.,(left, neutral, right)]"
parameter_04 = "Parameter 04: [Body Height , Controls the height of the robot's torso. If set to ‘low’ robot lowers it’s body to the ground. If set to ‘neutral’ robot dog maintains a neutral body height. If set to ‘high’ robot dog raises it’s torso higher.,(low, neutral, high)]"
parameter_05 = "Parameter 05: [Body Direction, Controls whether the robot faces the user or a relevant object in the scene. If set to ‘user’ robot faces the user. If set to ‘object’ robot faces the relevant object (the strawberry) in the scene,(user, object)]"
parameter_06 = "Parameter 06: [Pose Duration, Controls the duration in which the robot holds its pose. If set to ‘short’ robot holds pose for a short length duration of 1 second. If set to ‘medium’ robot holds pose for a medium length duration of 4 seconds. If set to ‘long’ robot holds pose for a long length duration of 8 seconds.,(short, medium, long)]"
parameter_07 = "Parameter 07: [Motion Velocity, Controls the velocity of the robot as it moves to a given pose. If set to ‘slow’ robot moves at a slow speed. If set to ‘medium’ robot moves at a normal speed. If set to ‘fast’ robot moves at a high speed. ,(slow, medium, fast)]"
parameter_08 = "Parameter 08: [Motion Smoothness, Controls whether the robot's motion is smooth or shaky. If set to ‘smooth’ the robot’s motion is smooth without any disturbances. If set to ‘shaky’ the robot’s motion is shaky looks like it is trembling.,(smooth, shaky)]"

# This is done to randomize the order in which the parameters are presented to GPT, to eliminate any bias for params presented first
param_list = [parameter_01, parameter_02, parameter_03, parameter_04, parameter_05, parameter_06, parameter_07, parameter_08]
random.shuffle(param_list) # Comment this out to remove randomness 


def generate_gpt_user_prompt():
    
    return f'''
Consider a scenario where you are collaborating with a {robot_morphology} robot to locate and pick strawberries in a strawberry patch. 

This {robot_morphology} robot is in a state '{robot_state_name}'. In this state, {robot_state_description}.

This robot uses its body pose to express its internal state. Below is a list of eight (8) motion parameters, complete with a description and a value range for each of those parameters. These parameters govern the characteristics of the robot's pose. The data in this list is in the format: [Parameter Name, Parameter Description, (Value Range)]

{param_list[0]}

{param_list[1]}

{param_list[2]}

{param_list[3]}

{param_list[4]}

{param_list[5]}

{param_list[6]}

{param_list[7]}


Your Task:
From the available eight (8) motion parameters, please select at minimum three and a maximum of six of the most relevant parameters to express the robot state '{robot_state_name}'. Please include reasonable values within the provided value ranges for each selected motion parameter. Please only select those that you believe are most relevant. Please include no additional text or explanation.

Please include no additional text or explanation. There should be no blank lines

Your response must keep the selected parameters in numerical order, and be in this exact format with the exact spacing shown below (one line for every selected parameter):

[##, Z], (Parameter Name, Value)
[##, Z], (Parameter Name, Value)
...
[##, Z], (Parameter Name, Value)

## = parameter number (e.g. 04)
Z = value option, either A, B, or C. Some parameters only have two options (A, B) instead of 3 options (A, B, C) 
Parameter Name = name of the selected parameter
Value = value selected for that parameter
'''


gpt_user_prompt = generate_gpt_user_prompt()

print(f'Your prompt for GPT is: \n\n{gpt_assistant_prompt}\n{gpt_user_prompt}')

Your prompt for GPT is: 

You are an expert roboticist and understand how to design communicative expressions for human-robot interaction.

Consider a scenario where you are collaborating with a dog-shaped quadruped robot to locate and pick strawberries in a strawberry patch. 

This dog-shaped quadruped robot is in a state 'needs help'. In this state, the robot is experiencing an error and needs help from the user.

This robot uses its body pose to express its internal state. Below is a list of eight (8) motion parameters, complete with a description and a value range for each of those parameters. These parameters govern the characteristics of the robot's pose. The data in this list is in the format: [Parameter Name, Parameter Description, (Value Range)]

Parameter 02: [Body Lean, Controls the forwards-backwards lean angle of the robot's torso. If set to ‘backwards’ robot’s torso leans backwards. If set to ‘neutral’ robot’s torso remains unchanged. If set to ‘forwards’ robot’s torso le

### Now build the OpenAI API Request

### Multi-Iteration

In [16]:
attempt_ID = '03'
sheet_name = robot_state_name + ' ' + attempt_ID 

# Enter the data in spreadsheet format
workbook_path = "./../data/part1_collection/llm_generated_data.xlsx"
response_book = load_workbook(workbook_path)

try: # Try to open existing sheet
    response_sheet = response_book.get_sheet_by_name(sheet_name)
except KeyError:  # If ot doesn't exist. create it
    response_sheet = response_book.create_sheet(title=sheet_name)
response_sheet["B1"] = "state"
response_sheet["C1"] = "iteration"
response_sheet["D1"] = "P1 Tilt"
response_sheet["E1"] = "P2 Lean"
response_sheet["F1"] = "P3 Turn"
response_sheet["G1"] = "P4 Body Height"
response_sheet["H1"] = "P5 Direction"
response_sheet["I1"] = "P6 Duration"
response_sheet["J1"] = "P7 Velocity"
response_sheet["K1"] = "P8 Smoothness"

response_book.save(workbook_path)

/var/folders/vx/sqc7hy3d7tzbzb4vn45c7n8h0000gn/T/ipykernel_4006/575319913.py:9: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  response_sheet = response_book.get_sheet_by_name(sheet_name)


In [17]:
print(f'Robot State: {robot_state_name}\n')

selected_param_value_pairs = []

for iteration in range(0, 80):
    print(f'~~~~~~~~~~~~ Iteration {iteration:02d}')

    # adding to excel
    response_sheet["B"+str(iteration+2)] = robot_state_name
    response_sheet["C"+str(iteration+2)] = iteration

    
    # calling GPT client
    completion = client.chat.completions.create(
        model="gpt-4",  # "gpt-3.5-turbo" (use this for dev/testing) or "gpt-4" (use this when deployed, more expensive)
        messages=[
            {"role": "system", "content": gpt_assistant_prompt},
            {"role": "user", "content": generate_gpt_user_prompt()}],
        temperature=1.0,         # 0.2 or 0.5
        max_tokens=500,
        frequency_penalty= 1.0   # 0.0 or 0.2
    )

    # Printing result from call to GPT client
    print(completion.choices[0].message.content, '\n\n')

    
    # Now iterate and count the responses
    for line in completion.choices[0].message.content.split('\n'):
        
        # Check if the line is blank or too short, skip if it is
        if len(line) < 6:
            continue  # Skip to the next iteration if line is blank or too short

        try:

            collected_identifier = line[1:3]+line[5] 
            print(f'Appending: {collected_identifier}')
            selected_param_value_pairs.append((collected_identifier))

            if collected_identifier[0:2] == '01':
                response_sheet["D"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '02':
                response_sheet["E"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '03':
                response_sheet["F"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '04':
                response_sheet["G"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '05':
                response_sheet["H"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '06':
                response_sheet["I"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '07':
                response_sheet["J"+str(iteration+2)] = collected_identifier[2]

            elif collected_identifier[0:2] == '08':
                response_sheet["K"+str(iteration+2)] = collected_identifier[2]
        
            #old# print(f'Appending: {line[1:3]+line[5]}')
            #old# selected_param_value_pairs.append((line[1:3]+line[5]))
        
        except IndexError:
                print(f"Skipping line due to formatting issues: {line}")
                continue  # Skip the line if there's an IndexError (unexpected format)


    print('\n')

print(f'selected_param_value_pairs\nLength: {len(selected_param_value_pairs)}\n{selected_param_value_pairs}')

response_book.save(workbook_path)


Robot State: needs help

~~~~~~~~~~~~ Iteration 00
[02, A], (Body Lean, backwards)
[03, B], (Body Turn, neutral)
[04, A], (Body Height, low)
[05, A], (Body Direction, user)
[08, B], (Motion Smoothness, shaky) 


Appending: 02A
Appending: 03B
Appending: 04A
Appending: 05A
Appending: 08B


~~~~~~~~~~~~ Iteration 01
[01, A], (Body Tilt, left)
[02, A], (Body Lean, backwards)
[03, B], (Body Turn, neutral)
[04, A], (Body Height , low)
[05, A], (Body Direction, user) 
[08, B], (Motion Smoothness, shaky) 


Appending: 01A
Appending: 02A
Appending: 03B
Appending: 04A
Appending: 05A
Appending: 08B


~~~~~~~~~~~~ Iteration 02
[01, B], (Body Tilt, neutral)
[02, A], (Body Lean, backwards)
[03, B], (Body Turn, neutral)
[05, A], (Body Direction, user)
[06, C], (Pose Duration, long)
[08, B], (Motion Smoothness, shaky) 


Appending: 01B
Appending: 02A
Appending: 03B
Appending: 05A
Appending: 06C
Appending: 08B


~~~~~~~~~~~~ Iteration 03
[01, A], (Body Tilt, left)
[02, C], (Body Lean, forwards)
[03, B]

In [18]:
# Parse the collected data
def pair_parser(pv_pairs, printer=None):

    param01_valA = 0
    param01_valB = 0
    param01_valC = 0
    param02_valA = 0
    param02_valB = 0
    param02_valC = 0
    param03_valA = 0
    param03_valB = 0
    param03_valC = 0
    param04_valA = 0
    param04_valB = 0
    param04_valC = 0
    param05_valA = 0
    param05_valB = 0
    param06_valA = 0
    param06_valB = 0
    param06_valC = 0
    param07_valA = 0
    param07_valB = 0
    param07_valC = 0
    param08_valA = 0
    param08_valB = 0
    
    for pv_pair in pv_pairs:
        if pv_pair[1] == '1':
            if pv_pair[2] == 'A':
                param01_valA +=1
            if pv_pair[2] == 'B':
                param01_valB +=1
            if pv_pair[2] == 'C':
                param01_valC +=1
        if pv_pair[1] == '2':
            if pv_pair[2] == 'A':
                param02_valA +=1
            if pv_pair[2] == 'B':
                param02_valB +=1
            if pv_pair[2] == 'C':
                param02_valC +=1
        if pv_pair[1] == '3':
            if pv_pair[2] == 'A':
                param03_valA +=1
            if pv_pair[2] == 'B':
                param03_valB +=1
            if pv_pair[2] == 'C':
                param03_valC +=1
        if pv_pair[1] == '4':
            if pv_pair[2] == 'A':
                param04_valA +=1
            if pv_pair[2] == 'B':
                param04_valB +=1
            if pv_pair[2] == 'C':
                param04_valC +=1
        if pv_pair[1] == '5':
            if pv_pair[2] == 'A':
                param05_valA +=1
            if pv_pair[2] == 'B':
                param05_valB +=1
        if pv_pair[1] == '6':
            if pv_pair[2] == 'A':
                param06_valA +=1
            if pv_pair[2] == 'B':
                param06_valB +=1
            if pv_pair[2] == 'C':
                param06_valC +=1
        if pv_pair[1] == '7':
            if pv_pair[2] == 'A':
                param07_valA +=1
            if pv_pair[2] == 'B':
                param07_valB +=1
            if pv_pair[2] == 'C':
                param07_valC +=1
        if pv_pair[1] == '8':
            if pv_pair[2] == 'A':
                param08_valA +=1
            if pv_pair[2] == 'B':
                param08_valB +=1

    # Sanity check
    sum = param01_valA + param02_valA + param03_valA + param04_valA + param05_valA + param06_valA + param07_valA + param08_valA + param01_valB + param02_valB + param03_valB + param04_valB + param05_valB + param06_valB + param07_valB + param08_valB + param01_valC + param02_valC + param03_valC + param04_valC + param06_valC + param07_valC
    original_length = len(pv_pairs)    
    sanity_check = sum == original_length
    
    if printer:
        print(f'sanity check: {sanity_check}\n')
        print('param01_valA =', param01_valA)
        print('param01_valB =', param01_valB)
        print('param01_valC =', param01_valC)
        print('param02_valA =', param02_valA)
        print('param02_valB =', param02_valB)
        print('param02_valC =', param02_valC)
        print('param03_valA =', param03_valA)
        print('param03_valB =', param03_valB)
        print('param03_valC =', param03_valC)
        print('param04_valA =', param04_valA)
        print('param04_valB =', param04_valB)
        print('param04_valC =', param04_valC)
        print('param05_valA =', param05_valA)
        print('param05_valB =', param05_valB)
        print('param06_valA =', param06_valA)
        print('param06_valB =', param06_valB)
        print('param06_valC =', param06_valC)
        print('param07_valA =', param07_valA)
        print('param07_valB =', param07_valB)
        print('param07_valC =', param07_valC)
        print('param08_valA =', param08_valA)
        print('param08_valB =', param08_valB)

In [19]:
pair_parser(selected_param_value_pairs, printer=True)

sanity check: False

param01_valA = 28
param01_valB = 25
param01_valC = 2
param02_valA = 29
param02_valB = 27
param02_valC = 23
param03_valA = 20
param03_valB = 35
param03_valC = 4
param04_valA = 48
param04_valB = 4
param04_valC = 10
param05_valA = 53
param05_valB = 1
param06_valA = 0
param06_valB = 2
param06_valC = 31
param07_valA = 19
param07_valB = 0
param07_valC = 1
param08_valA = 0
param08_valB = 68


### Single Iteration

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",  # "gpt-3.5-turbo" (use this for dev/testing) or "gpt-4" (use this when deployed, more expensive)
    messages=[
        {"role": "system", "content": gpt_assistant_prompt},
        {"role": "user", "content": gpt_user_prompt}],
    temperature=0.5,
    max_tokens=500,
    frequency_penalty=0
)

print('Raw Model Output:\n\n')
print(completion.choices[0].message)

In [ ]:
print(f'Robot State: {robot_state_name}\n')

print(completion.choices[0].message.content)